In [1]:
import gym
import random

In [2]:
env = gym.make('Breakout-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n #number of actios we can take

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [10]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice(list(range(actions)))
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score: {}'.format(episode, score))
env.close()

Episode:0 Score: 3.0
Episode:1 Score: 0.0
Episode:2 Score: 3.0
Episode:3 Score: 0.0
Episode:4 Score: 0.0
Episode:5 Score: 2.0
Episode:6 Score: 1.0
Episode:7 Score: 1.0
Episode:8 Score: 4.0
Episode:9 Score: 2.0


In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [13]:
#https://arxiv.org/pdf/1312.5602v1.pdf architecture based on DeepMind paper
def create_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(16, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(32, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [19]:
del model

In [20]:
model = create_model(height, width, channels, actions)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 16)     3088      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 32)     8224      
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               10617088  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 10,645,108
Trainable params: 10,645,108
Non-trainable params: 0
____________________________________________

In [16]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [17]:
def create_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = create_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  276/10000: episode: 1, duration: 4.109s, episode steps: 276, steps per second:  67, episode reward:  2.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 1.591 [0.000, 3.000],  loss: --, mean_q: --, mean_eps: --
  457/10000: episode: 2, duration: 2.564s, episode steps: 181, steps per second:  71, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.348 [0.000, 3.000],  loss: --, mean_q: --, mean_eps: --
  707/10000: episode: 3, duration: 3.781s, episode steps: 250, steps per second:  66, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.456 [0.000, 3.000],  loss: --, mean_q: --, mean_eps: --
  945/10000: episode: 4, duration: 3.561s, episode steps: 238, steps per second:  67, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 1.445 [0.000, 3.000],  loss: --, mea

In [30]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


c:\users\filip\appdata\local\programs\python\python38\lib\site-packages\gym\envs\atari\environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1: reward: 145.000, steps: 705
Episode 2: reward: 260.000, steps: 985
Episode 3: reward: 140.000, steps: 782
Episode 4: reward: 110.000, steps: 689
Episode 5: reward: 285.000, steps: 1423
Episode 6: reward: 185.000, steps: 845
Episode 7: reward: 110.000, steps: 681
Episode 8: reward: 25.000, steps: 375
Episode 9: reward: 210.000, steps: 817
Episode 10: reward: 150.000, steps: 663
162.0


In [32]:
dqn.save_weights('savedModels/10k-weights.h5f')

[WARNING] savedModels/10k-weights.h5f.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [ ]:
dqn.load_weights('savedModels/10k-weights.h5f')